# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np

In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("../data/wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv", parse_dates=['date'])

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [3]:
train.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0


In [4]:
test.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-09-10,640.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-09-10,5609.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-09-10,3121.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-09-10,362.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-09-10,533.0


In [5]:
min(test['date']), max(test['date'])

(Timestamp('2016-09-10 00:00:00'), Timestamp('2016-11-10 00:00:00'))

In [6]:
min(train['date']), max(train['date'])

(Timestamp('2016-01-01 00:00:00'), Timestamp('2016-08-31 00:00:00'))

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [7]:
### Your code here
from dateutil.relativedelta import relativedelta
def create_validation(df, start_date):
    return df[(df['date'] >= pd.to_datetime(start_date) - relativedelta(months=3)) & \
              (df['date'] <  pd.to_datetime(start_date) - relativedelta(months=0))].index, \
           df[(df['date'] >= pd.to_datetime(start_date) - relativedelta(months=0)) & \
              (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=2))].index

In [21]:
### Your validation iterator
CVIterator = []
for i in range(3):
    date = '2016-0%d-10' % (i + 4)
    print(date)
    CVIterator.append(create_validation(train, date))

2016-04-10
2016-05-10
2016-06-10


In [9]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [16]:
def last_15_days_pred(train_df, val_df):
    last_day = max(train_df['date'])
    
    select = (train_df['date'] > (last_day - relativedelta(days=15)))
    train_df = train_df.loc[select].copy()
    train_df.drop('date', axis=1, inplace=True)
    median_baseline = train_df.groupby(['Page']).median().reset_index()
    median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)
    
    return val_df.merge(median_baseline, on="Page", how='left')

In [17]:
def validate_smape(CVIterator, df_predictor):
    scores = []
    for train_indexes, val_indexes in CVIterator:
        val_df_with_pred = df_predictor(train.iloc[train_indexes].copy(), 
                                        train.iloc[val_indexes].copy())
        score = pandas_smape(val_df_with_pred)
        scores.append(score)
    return scores, np.mean(scores)

In [18]:
def test_smape(df_predictor):
    test_df_with_pred = df_predictor(train.copy(), 
                                     test.copy())
    return pandas_smape(test_df_with_pred)

In [19]:
validate_smape(CVIterator, last_15_days_pred)

([39.968622328046635, 40.848838633224524, 42.603892486759456],
 41.14045114934354)

In [20]:
test_smape(last_15_days_pred)

40.007012702044406

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [22]:
train = pd.read_csv('../data/train_french.csv')
test = pd.read_csv('../data/test_french.csv')

In [23]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [24]:
test.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,6029,5,OUI,500,Famille,agent; employe,Maison,1,7,C,...,0,164,41,0,0,1,NON,30,1,107.94
1,62128,2,NON,0,Famille,enseignant; formateur; chercheur,Appartement,1,1,D,...,1,85,100,0,0,1,NON,30,2,80.41
2,81065,4,NON,1000,Indivividu,homme ou femme au foyer,Sans information,2,99,~,...,0,94,100,0,0,1,NON,30,2,142.12
3,5023,3,NON,500,Indivividu,agent; employe,Maison,2,38,C,...,0,101,100,0,0,5,NON,30,2,76.55
4,62178,2,NON,500,Famille,enseignant; formateur; chercheur,Appartement,1,1,~,...,1,125,100,1,0,1,NON,0,2,150.45


In [25]:
import pandas_profiling as pp

In [26]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [27]:
pp.ProfileReport(test)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [58]:
def mape_score(y_true, y_pred):
    return np.sum(100. / y_true.shape[0] * np.abs((y_true - y_pred) / y_true))

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
x_train, x_val, y_train, y_val = train_test_split(train.drop('prime_tot_ttc', axis=1), 
                                                   train['prime_tot_ttc'],
                                                   test_size=0.1, 
                                                   random_state=0xCAFFE)

In [66]:
from catboost import CatBoostRegressor

In [67]:
categorical_features_indices = np.where(x_train.dtypes != np.float)[0]

In [82]:
model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.6, loss_function='MAE')

In [83]:
model.fit(x_train, y_train, 
          cat_features=categorical_features_indices,
          eval_set=(x_val, y_val),
          plot=True)

0:	learn: 96.2024435	test: 96.1577554	best: 96.1577554 (0)	total: 97.3ms	remaining: 1m 37s
1:	learn: 95.9024535	test: 95.8577654	best: 95.8577654 (1)	total: 183ms	remaining: 1m 31s
2:	learn: 95.6024635	test: 95.5577754	best: 95.5577754 (2)	total: 250ms	remaining: 1m 23s
3:	learn: 95.3024735	test: 95.2577854	best: 95.2577854 (3)	total: 288ms	remaining: 1m 11s
4:	learn: 95.0024835	test: 94.9577954	best: 94.9577954 (4)	total: 326ms	remaining: 1m 4s
5:	learn: 94.7024935	test: 94.6578054	best: 94.6578054 (5)	total: 372ms	remaining: 1m 1s
6:	learn: 94.4025035	test: 94.3578154	best: 94.3578154 (6)	total: 415ms	remaining: 58.9s
7:	learn: 94.1025134	test: 94.0578254	best: 94.0578254 (7)	total: 451ms	remaining: 56s
8:	learn: 93.8025234	test: 93.7578354	best: 93.7578354 (8)	total: 490ms	remaining: 53.9s
9:	learn: 93.5025334	test: 93.4578454	best: 93.4578454 (9)	total: 521ms	remaining: 51.5s
10:	learn: 93.2025434	test: 93.1578553	best: 93.1578553 (10)	total: 557ms	remaining: 50.1s
11:	learn: 92.90

In [84]:
y_val_pred = model.predict(x_val)
y_test_pred = model.predict(test.drop('prime_tot_ttc', axis=1))

In [85]:
mape_score(np.array(y_val), y_val_pred)

8.240488550115176

In [86]:
mape_score(np.array(test['prime_tot_ttc']), y_test_pred)

8.801507461987192

In [87]:
test['prime_tot_ttc'].head()

0    107.94
1     80.41
2    142.12
3     76.55
4    150.45
Name: prime_tot_ttc, dtype: float64

In [88]:
y_test_pred[:5]

array([113.90014761,  76.31716433, 129.60616102,  75.0996137 ,
       110.91885543])